In [165]:
import urllib
import requests
import json
import folium

#Definition of parameters
text = input("Please enter an address: ") #Input of the address to search for
base_url="https://api3.geo.admin.ch/rest/services/api/SearchServer?"
parameters = {"searchText": text, "origins": "address", "type": "locations",}

#Request
r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

data = json.loads(r.content)



# Create a map centered at the first item's latitude and longitude
map_center = [data['results'][0]['attrs']['lat'], data['results'][0]['attrs']['lon']]
m = folium.Map(location=map_center, zoom_start=20)

# Add markers for each item with annotations
for i, result in enumerate(data['results']):
    lat = result['attrs']['lat']
    lon = result['attrs']['lon']
    label = f"Index: {i}"
    folium.Marker([lat, lon], popup=label).add_to(m)

# Display the map
m

In [166]:
import pyproj
#Select the desired item
index = int(input("Please enter the index of the item you want to select: "))
selected_building = data['results'][index]

lon = float(selected_building['attrs']['lon'])
lat = float(selected_building['attrs']['lat'])

print(f"Selected building: {selected_building['attrs']['label']}, [{lat}, {lon}]")

# Define the WGS84 and LV95+ coordinate systems
wgs84 = pyproj.CRS("EPSG:4326")
lv95 = pyproj.CRS("EPSG:2056")

# Create a transformer object
transformer_toLV95 = pyproj.Transformer.from_crs(wgs84, lv95, always_xy=True)
transformer_toWGS84 = pyproj.Transformer.from_crs(lv95, wgs84, always_xy=True)

# Convert lon and lat from WGS84 to LV95+
lv95_lon, lv95_lat = transformer_toLV95.transform(lon, lat)

# Print the converted coordinates
print(f"LV95+ coordinates: [{lv95_lon}, {lv95_lat}]")


Selected building: Widmergasse 19 <b>8466 Trüllikon</b>, [47.63404846191406, 8.694016456604004]
LV95+ coordinates: [2694348.4256816725, 1276686.3835739407]


In [167]:
from openrouteservice import client

ors_api_key = "5b3ce3597851110001cf62487accec8762334d3691b56d17827eb48b"

ors = client.Client(key=ors_api_key)
location = (lon, lat)
params_iso = {'profile': 'driving-car', 'range':[30*60], 'locations': [[lon,lat]]}

iso = ors.isochrones(**params_iso)

n = folium.Map(tiles='openstreetmap', location=[lat, lon], zoom_start=10)

folium.features.GeoJson(iso).add_to(n)
folium.map.Marker(location).add_to(n)

n

#print(iso)

In [168]:
import pandas as pd
import pandas as pd

gwr0=pd.read_csv("GWR\\gebaeude_batiment_edificio.csv", sep="	", encoding="utf-8")[['EGID','GDEKT','GGDENR','GGDENAME','EGRID','LGBKR','LPARZ','LPARZSX','LTYP','GEBNR','GBEZ','GKODE','GKODN','GKSCE','GSTAT','GKAT','GKLAS','GBAUJ','GBAUM','GBAUP','GABBJ','GAREA','GVOL','GVOLNORM','GVOLSCE','GASTW','GANZWHG','GAZZI','GSCHUTZR','GEBF','GWAERZH1','GENH1','GWAERSCEH1','GWAERDATH1','GWAERZH2','GENH2','GWAERSCEH2','GWAERDATH2','GWAERZW1','GENW1','GWAERSCEW1','GWAERDATW1','GWAERZW2','GENW2','GWAERSCEW2','GWAERDATW2','GEXPDAT']]

# List of years to exclude
exclude_years = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
exclude_ids = ['7400','7410', '7411', '7420', '7421','7460','7461']

# Filter the gwr dataframe based on the condition
gwr1 = gwr0.loc[~gwr0['GWAERDATH1'].str.split('-').str[0].isin(exclude_years)].copy()
gwr2= gwr1.loc[~gwr1['GWAERZH1'].isin(exclude_ids)].copy()
gwr2['LON'], gwr2['LAT'] = transformer_toWGS84.transform(gwr2['GKODE'], gwr2['GKODN'])

gwr=gwr2
gwr.head()

C:\Users\carlo\AppData\Local\Temp\ipykernel_29280\814887841.py:4: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  gwr0=pd.read_csv("GWR\\gebaeude_batiment_edificio.csv", sep="	", encoding="utf-8")[['EGID','GDEKT','GGDENR','GGDENAME','EGRID','LGBKR','LPARZ','LPARZSX','LTYP','GEBNR','GBEZ','GKODE','GKODN','GKSCE','GSTAT','GKAT','GKLAS','GBAUJ','GBAUM','GBAUP','GABBJ','GAREA','GVOL','GVOLNORM','GVOLSCE','GASTW','GANZWHG','GAZZI','GSCHUTZR','GEBF','GWAERZH1','GENH1','GWAERSCEH1','GWAERDATH1','GWAERZH2','GENH2','GWAERSCEH2','GWAERDATH2','GWAERZW1','GENW1','GWAERSCEW1','GWAERDATW1','GWAERZW2','GENW2','GWAERSCEW2','GWAERDATW2','GEXPDAT']]


,EGID,GDEKT,GGDENR,GGDENAME,EGRID,LGBKR,LPARZ,LPARZSX,LTYP,GEBNR,...,GENW1,GWAERSCEW1,GWAERDATW1,GWAERZW2,GENW2,GWAERSCEW2,GWAERDATW2,GEXPDAT,LON,LAT
1,11513433,ZH,1,Aeugst am Albis,CH587820017717,0.0,1543.0,NaN,NaN,1198,...,7560.0,860.0,2002-09-05,NaN,NaN,NaN,NaN,2024-04-27,8.504400,47.278398
2,11517090,ZH,1,Aeugst am Albis,CH707701872012,0.0,1361.0,NaN,NaN,1017,...,7530.0,860.0,2002-09-05,NaN,NaN,NaN,NaN,2024-04-27,8.482808,47.268628
6,1600003,ZH,1,Aeugst am Albis,CH807701712026,0.0,1187.0,NaN,NaN,57,...,7560.0,860.0,2001-11-29,NaN,NaN,NaN,NaN,2024-04-27,8.491615,47.283789
7,1600005,ZH,1,Aeugst am Albis,CH887120017707,0.0,1186.0,NaN,NaN,35,...,7560.0,860.0,2001-11-29,NaN,NaN,NaN,NaN,2024-04-27,8.492827,47.283850
8,1600006,ZH,1,Aeugst am Albis,CH827771200185,0.0,1189.0,NaN,NaN,61,...,7560.0,860.0,2001-11-29,NaN,NaN,NaN,NaN,2024-04-27,8.492247,47.283976


In [169]:
from shapely.geometry import Point, Polygon

# Extract the coordinates of the isochrone polygon
coordinates = Polygon(iso['features'][0]['geometry']['coordinates'][0])

# Create a list of points from the GWR data
points = [Point(xy) for xy in zip(gwr['LON'], gwr['LAT'])] 

# Create a list of boolean values indicating whether the points are within the isochrone polygon
is_within = [coordinates.contains(point) for point in points]

# Filter the GWR data to include only the points within the isochrone polygon
gwr_within = gwr.loc[is_within].copy()

print(gwr_within.shape)
gwr_within.head()


(83674, 49)


,EGID,GDEKT,GGDENR,GGDENAME,EGRID,LGBKR,LPARZ,LPARZSX,LTYP,GEBNR,...,GENW1,GWAERSCEW1,GWAERDATW1,GWAERZW2,GENW2,GWAERSCEW2,GWAERDATW2,GEXPDAT,LON,LAT
20645,191967381,ZH,22,Benken (ZH),CH987912537761,0.0,1520,NaN,NaN,393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-27,8.637575,47.659032
20646,192002494,ZH,22,Benken (ZH),CH515370127797,0.0,1815,NaN,NaN,6.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-27,8.656595,47.657582
20647,192029645,ZH,22,Benken (ZH),CH245312777492,0.0,2238,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-27,8.655502,47.652113
20650,200295849,ZH,22,Benken (ZH),CH757012775382,0.0,1701,NaN,NaN,406,...,7560.0,869.0,2005-10-05,NaN,NaN,NaN,NaN,2024-04-27,8.648838,47.656050
20651,200299009,ZH,22,Benken (ZH),CH531277537978,0.0,1654,NaN,NaN,407,...,7598.0,869.0,2005-10-05,NaN,NaN,NaN,NaN,2024-04-27,8.658091,47.656577


In [171]:
import psycopg2
from tqdm import tqdm

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="egm",
    user="postgres",
    password="password"
)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Execute SQL queries
cursor.execute("SELECT Find_SRID('public', 'gesteinsdichte', 'geom'), Find_SRID('public', 'erdbebenzonen', 'geom'), Find_SRID('public', 'lithologie', 'geom'),Find_SRID('public', 'grundwasser', 'geom');")
rows = cursor.fetchall()
for row in rows:
    print(row)

gwr_within['gesteinsdichte'] = None
gwr_within['erdbebenzonen'] = None
gwr_within['lithologie'] = None
gwr_within['grundwasser'] = None

tablename = ['gesteinsdichte','erdbebenzonen', 'lithologie', 'grundwasser']

for index, building in tqdm(gwr_within.iterrows()):
    point = (float(building['GKODE']), float(building['GKODN']), 2056)
    for table in tablename:
        if table == 'gesteinsdichte':
            cursor.execute(f"SELECT rhob_m FROM {table} WHERE ST_CONTAINS(geom, ST_POINT{point});")
        elif table == 'erdbebenzonen':
            cursor.execute(f"SELECT zone FROM {table} WHERE ST_CONTAINS(geom, ST_POINT{point});")
        elif table == 'lithologie':
            cursor.execute(f"SELECT gesteinsty FROM {table} WHERE ST_CONTAINS(geom, ST_POINT{point});")
        elif table == 'grundwasser':
            cursor.execute(f"SELECT produktiv FROM {table} WHERE ST_CONTAINS(geom, ST_POINT{point});")
        rows = cursor.fetchall()
        for row in rows:
            gwr_within.at[index, table]=row[0]

# Close the cursor and connection
cursor.close()
conn.close()


(2056, 2056, 2056, 2056)


83674it [3:07:58,  7.42it/s]


In [188]:
for building in gwr_within.iterrows():
    if building[1]['gesteinsdichte'] > 1700 and building[1]['gesteinsdichte']<2800 and building[1]['erdbebenzonen'] in ['Z1a', 'Z1b', 'Z2'] and building[1]['lithologie']=="Festgestein" and building[1]['grundwasser'] in ["Nutzbare Grundwassermaechtigkeit 2 -10 m", "Nutzbare Grundwassermaechtigkeit ueber 20 m"]:
        folium.map.Marker([building[1]['LAT'], building[1]['LON']], popup=building[1]['EGID']).add_to(n)

n
    